# FugueSQL Syntax


The `FugueSQL` syntax is between standard SQL, JSON, and Python. The goals are
* To be fully compatible with standard `SQL SELECT` statement
* To minimize syntax overhead, to make code as short as possible while still easy to read
* Allow users to fully describe their compute logic in SQL as opposed to Python

To achieve these goals, enhancements were made to the standard SQL syntax that will be demonstrated here.

## Hello World

First, we start with the basic syntax `FugueSQL`. We import `FugueSQLWorkflow`, which is based off of `FugueWorkflow`. Fugue has both a programming interface, and SQL interface which have equivalent functionality. `Fugue` constructs a `Directed Acyclic Graph (DAG)` and evaluates it lazily. The `DAG` allows for compile-time validation, to ensure that incorrect code fails quickly. This is more important when Fugue is used with **Spark** and **Dask**. Catching failures quickly on these large computation environments saves users from expensive failed program runs. 

In [1]:
from fugue_sql import FugueSQLWorkflow

with FugueSQLWorkflow() as dag:
    dag("""
    CREATE [[0,"hello"],[1,"world"]] SCHEMA number:int,word:str
    PRINT
    """)

ArrayDataFrame
number:int|word:str
----------+--------
0         |hello   
1         |world   
Total count: 2



The `CREATE` keyword here is a Fugue keyword. We'll dive into [extensions](..extensions.ipynb) later and learn more about integrating Python functions into FugueSQL.

## SQL Compliant

All standard SQL keywords are available in Fugue SQL. In this example, `GROUP BY`, `WHERE`, `SELECT`, `FROM` are all the same as standard SQL.

In [2]:
# Defining data
data = [
    ["A", "2020-01-01", 10],
    ["A", "2020-01-02", None],
    ["A", "2020-01-03", 30],
    ["B", "2020-01-01", 20],
    ["B", "2020-01-02", None],
    ["B", "2020-01-03", 40]
]
schema = "id:str,date:date,value:double"

with FugueSQLWorkflow() as dag:
    df = dag.df(data, schema)
    dag("""
    SELECT id, date, MIN(value) value
    FROM df
    WHERE value > 20
    GROUP BY id
    PRINT
    """)

PandasDataFrame
id:str|date:str                                                                        |value:double
------+--------------------------------------------------------------------------------+------------
A     |2020-01-03 00:00:00.000000                                                      |30.0        
B     |2020-01-03 00:00:00.000000                                                      |40.0        
Total count: 2



Note that the DataFrame `df` was accessed inside the SQL expression. Fugue DataFrames defined are automatically accessible by the DAG. Other variables need to be passed in through [Jinja templating](../syntax.ipynb). More on this will be shown when we explore how Python and FugueSQL interact.

## Input and Output

Instead of creating the `DataFrame` inside the `FugueSQLWorkflow`, actual data work often require loading in the `DataFrame`. `Fugue` has two keywords in `SAVE` and `LOAD`. Using these allow SQL users to orchestrate their ETL jobs with SQL logic. A csv file can be loaded in, transformed, and then saved elsewhere.


In [3]:
with FugueSQLWorkflow() as dag:
    dag("""
    CREATE [[0,"1"]] SCHEMA a:int,b:str
    SAVE OVERWRITE "/tmp/f.parquet"
    SAVE OVERWRITE "/tmp/f.csv" (header=true)
    SAVE OVERWRITE "/tmp/f.json"
    SAVE OVERWRITE PARQUET "/tmp/f"
    """)
    dag("""
    LOAD "/tmp/f.parquet" PRINT
    LOAD "/tmp/f.parquet" COLUMNS a PRINT
    LOAD PARQUET "/tmp/f" PRINT
    LOAD "/tmp/f.csv" (header=true) PRINT
    LOAD "/tmp/f.csv" (header=true) COLUMNS a:int,b:str PRINT
    LOAD "/tmp/f.json" PRINT
    LOAD "/tmp/f.json" COLUMNS a:int,b:str PRINT
    """)

_1 _State.RUNNING -> _State.FAILED  path 'C:\tmp\f.parquet' contains invalid characters


InvalidCharsInPath: path 'C:\tmp\f.parquet' contains invalid characters

json, csv, and parquet are support file formats. There are plans to support avro. Notice that parameters can be passed. If running on the default [execution engine](../execution_engine.ipynb), these would be passed on to **Pandas** `read_csv` and `to_csv`.  The file extension is used as a hint to use the appropriate load/save function. If the extension is not present in the filename, it has to be specified.

## Variable Assignment

From here, it should be getting clear that `Fugue` extends SQL in order to make it a more complete language. One of the additional features is variable assignment. Along with this, multiple `SELECT` statements can be used. This is the equivalent of temp tables or Common Table Expressions (CTE) in SQL.

In [ ]:
with FugueSQLWorkflow() as dag:
    df = dag.df([[0,"hello"],[1,"world"]],"number:int,word:str")
    dag("""
    SELECT * FROM df
    SAVE OVERWRITE "/tmp/f.csv"(header=true)

    a = LOAD "/tmp/f.csv" (header=true)
    temp = SELECT * FROM a WHERE number=1
    output = SELECT word FROM temp
    SAVE OVERWRITE "/tmp/output.csv"(header=true)

    new_a = LOAD "/tmp/output.csv"(header=true)
    PRINT new_a
    """)

## Execution Engine

So far, we've only dealt with the default [execution engine](../execution_engine.ipynb). If nothing is passed to the `FugueSQLWorkflow`, the `NativeExecutionEngine` is used. Similar to `Fugue` programming interface, the `execution engine` can easily be changed by passing it to `FugueSQLWorkflow`. Below is an example for Spark.

Take note of the output `DataFrame` in the example below. It will be a `SparkDataFrame`.

In [ ]:
from fugue_spark import SparkExecutionEngine
with FugueSQLWorkflow(SparkExecutionEngine) as dag:
    df = dag.df([[0,"hello"],[1,"world"]],"a:int,b:str")
    dag("""
    SELECT * FROM df WHERE a=0  # see we can use df directly defined outside
    PRINT
    """)

## Anonymity

In `FugueSQL`, one of the simplifications is anonymity. It’s optional, but it usually can significantly simplify your code and make it more readable.

For a statement that only needs to consume the previous dataframe, a `FROM` keyword is not needed. `PRINT` is the best example. `SAVE` is another example. This is can be applied to other keywords. In this example we'll use the `TAKE` function that just returns the number of rows specified.

In [4]:
with FugueSQLWorkflow() as dag:
    df = dag.df([[0,"hello"],[1,"world"]],"number:int,word:str")
    dag("""
    a = SELECT * FROM df
    TAKE 2 ROWS PRESORT number DESC       # a is consumed by TAKE

    b = SELECT * FROM df
    TAKE 2 ROWS FROM b PRESORT number DESC
    PRINT
    """)

PandasDataFrame
number:long|word:str
-----------+--------
1          |world   
0          |hello   
Total count: 2



## Inline Statements

The last enchancement is inline statements. One statement can be written in another in between `(` `)` . Anonymity and variable assignment often make this unneeded, but it's just good to know that this option exists.

In [5]:
with FugueSQLWorkflow() as dag:
    dag("""
    a = CREATE [[0,"hello"], [1,"world"]] SCHEMA number:int,word:str
    SELECT *
    FROM (TAKE 1 ROW FROM a)
    PRINT
    """)

PandasDataFrame
number:long|word:str
-----------+--------
0          |hello   
Total count: 1



## Multiple `dag` Calls

DataFrames in precending `dag` calls can be used in future `dag` calls. One use case of this is chaining Python programming along with `FugueSQL`.

In [6]:
with FugueSQLWorkflow() as dag:
    dag("""
    a=CREATE [[0,"hello"],[1,"world"]] SCHEMA number:int,word:str
    """)
    # This other_df will be joined in following dag
    other_df = dag.df([[0,"hello2"],[1,"world2"]],"number:int,word2:str")
    dag("""
    SELECT a.number num, other_df.word2 
    FROM a 
    INNER JOIN other_df
    ON a.number = other_df.number
    PRINT
    """)

PandasDataFrame
num:long|word2:str
--------+---------
0       |hello2   
1       |world2   
Total count: 2



This example above shows the possibility of combining Python and SQL workflows. This is useful if Python needs to connect to other place (AWS S3, Azure Blob Storage, Google Analytics) to retrieve data that is needed for the compute workflow. The data can be loaded in with Python and passed to `FugueSQLWorkflow`. 

In this tutorial we have gone through how to use standard SQL operations (and more) on top of Pandas, Spark, and Dask. We have also seen enhancements over standard SQL like anonymity and variable assignment.

In a [following section](python.ipynb) we'll look at more ways of integrating Python with `FugueSQLWorkflow` to extend the capabilities of using SQL.